In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from plotting_utils import *
import altair as alt
import altair_saver
alt.data_transformers.disable_max_rows()

## Control suite

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('finger_all_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('walker_walk_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bice_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
    load_jobs('bbe_all_v3*', subdir='bbe', title='SAC + BBE'),
    load_jobs('ufo_all_v2*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('ufo_all_v3*', subdir='exploration', title='SAC + DEEP'),
]
data = pd.concat(jobs, sort=False)
# data = data[(data.env == 'ball_in_cup') | (data.env == 'ball_in_cup_explore')]

# N = 500
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
# subset = subset[(subset['episode'] <= N)]

Ns = {
    'ball_in_cup': 500, 'ball_in_cup_explore': 500,
    'reacher': 500, 'reacher_explore': 500,
    'finger': 1000, 'finger_explore': 1000,
    'walker': 500, 'walker_explore': 500,
}

charts = []
for env in sorted(subset.env.unique()):
    N = Ns[env]
    chart = make_base_chart(
        subset[(subset.env == env) & (subset.episode <= N)], 
#         title=env, 
        title='', 
        color='title',
        window=20,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

    altair_saver.save(chart, f'results/neurips_{env}.pdf')
    charts.append(chart)
alt.vconcat(*charts)

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('finger_all_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('walker_walk_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bice_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
    load_jobs('bbe_all_v3*', subdir='bbe', title='SAC + BBE'),
    load_jobs('ufo_all_v2*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('ufo_all_v3*', subdir='exploration', title='SAC + DEEP'),
]
data = pd.concat(jobs, sort=False)

# Ns = {
#     'ball_in_cup': 500, 'ball_in_cup_explore': 500,
#     'reacher': 500, 'reacher_explore': 500,
#     'finger': 1000, 'finger_explore': 1000,
#     'walker': 500, 'walker_explore': 500,
# }

env_names = {
    'ball_in_cup': 'BIC', 'ball_in_cup_explore': 'BIC explore',
    'reacher': 'Reacher', 'reacher_explore': 'Reacher explore',
    'finger': 'Finger', 'finger_explore': 'Finger explore',
    'walker': 'Walker', 'walker_explore': 'Walker explore',
}

subsets = []
for env in sorted(data.env.unique()):
#     N = Ns[env] / 5
    N = 100    
    env_subset = data[(data.env == env) & (data.test == True) &
                      (data.episode <= N + 10) & (N - 10 < data.episode)].copy()
    env_subset['pretty_env'] = env_names[env]
    subsets.append(env_subset)
subset = pd.concat(subsets, sort=False)

chart = alt.Chart(subset, width=100, height=300).mark_bar(size=22).encode(
    x=alt.X('title', title='Algorithm', axis=None),
    y=alt.Y('mean(score)', title='Evaluation score', scale=alt.Scale(domain=(0,1000)), axis=alt.Axis(domainOpacity=0)),
    color=alt.Color('title', title='Algorithm', legend=alt.Legend(orient='none', legendX=950, legendY=0)),
    column=alt.Column('pretty_env', title='Environment',
                      sort=['BIC', 'BIC explore', 'Reacher', 'Reacher explore', 'Finger', 'Finger explore', 'Walker', 'Walker explore'],
#                       sort=['Walker', 'Walker explore'],
                      spacing=20, header=alt.Header(labelOrient='bottom', titleOrient='bottom', titleFontSize=24, labelFontSize=20)),
).configure_view(stroke='transparent')


# chart = alt.Chart(subset).mark_bar(opacity=1).encode(
#     column = alt.Column('env:O', spacing = 5, header = alt.Header(labelOrient = "bottom")),
#     x =alt.X('title', sort = ["SAC", "SAC + BBE", "SAC + DEX"],  axis=None),
#     y =alt.Y('mean(score):Q'),
#     color= alt.Color('title')
# ).configure_view(stroke='transparent')

# altair_saver.save(chart, 'results/control_suite_summary.pdf', method='node')
chart.display()

## Gridworld

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('grid40_warmstart_v6*no_explorationTrue*', root='/home/will/vlg5/exploration-reimplement', subdir='exploration', title='DDQN'),
    load_jobs('grid40_warmstart_v6*no_explorationFalse*', root='/home/will/vlg5/exploration-reimplement', subdir='exploration', title='DDQN + DEEP'),
#     load_jobs('grid40_warmstart_v6*bonus_scale0.1*', root='/home/will/vlg5/exploration-reimplement', subdir='bbe', title='DDQN + BBE, 0.1'),
    load_jobs('grid40_warmstart_v6*bonus_scale1', root='/home/will/vlg5/exploration-reimplement', subdir='bbe', title='DDQN + BBE'),    

]
data = pd.concat(jobs, sort=False)

N = 500
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset.episode = subset.episode - 20
subset = subset[(subset.episode > 0) & (subset.episode <= N)]

chart = make_base_chart(
    subset,
    title=f"",
    color='title',
    window=10,
)
chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
               y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,23))}, 
               color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

altair_saver.save(chart, f'results/grid40_warmstart.pdf')
chart

## Hallway

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('hallway_distractors_v13*no_explorationTrue*', subdir='exploration', title='SAC'),
    load_jobs('hallway_distractors_v13*no_explorationFalse*', subdir='exploration', title='SAC + DEEP'),
]
data = pd.concat(jobs, sort=False)

N = 300
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]

charts = []
for task in ['velocity_4_distractor']:
    chart = make_base_chart(
        subset[subset.task == task], 
        title=f"",
        color='title',
#         strokeDash='task',
#         detail='task',
        window=5,
    )
    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
               y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
               color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right'),
                               scale=alt.Scale(domain=['SAC', 'SAC + DEEP'], range=['#66c2a5', '#8da0cb'])))
    charts.append(chart)
    altair_saver.save(chart, f'results/hallway_{task}.pdf')
alt.hconcat(*charts)

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('hallway_distractors_v13*no_explorationTrue*', subdir='exploration', title='SAC'),
    load_jobs('hallway_distractors_v13*no_explorationFalse*', subdir='exploration', title='SAC + DEEP'),
]
data = pd.concat(jobs, sort=False)

N = 100
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]

charts = []
for task in ['velocity_4_inverse_distractor']:
    chart = make_base_chart(
        subset[subset.task == task], 
        title=f"",
        color='title',
#         strokeDash='task',
#         detail='task',
        window=5,
    )
    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
               y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
               color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right'),
                               scale=alt.Scale(domain=['SAC', 'SAC + DEEP'], range=['#66c2a5', '#8da0cb'])))
    charts.append(chart)
    altair_saver.save(chart, f'results/hallway_{task}.pdf')
alt.hconcat(*charts)

## Additional Control Suite

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('finger_all_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('walker_walk_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bice_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
    load_jobs('bbe_all_v3*', subdir='bbe', title='SAC + BBE'),
    load_jobs('ufo_all_v2*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('ufo_all_v3*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('sacqex_all_v2*', subdir='sac_qex', title='SAC 2Q'),
]
data = pd.concat(jobs, sort=False)
# data = data[(data.env == 'ball_in_cup') | (data.env == 'ball_in_cup_explore')]

# N = 500
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
# subset = subset[(subset['episode'] <= N)]

Ns = {
    'ball_in_cup': 500, 'ball_in_cup_explore': 500,
    'reacher': 500, 'reacher_explore': 500,
    'finger': 1000, 'finger_explore': 1000,
    'walker': 500, 'walker_explore': 500,
}

charts = []
for env in sorted(subset.env.unique()):
    N = Ns[env]
    chart = make_base_chart(
        subset[(subset.env == env) & (subset.episode <= N)], 
#         title=env, 
        title='', 
        color='title',
        window=20,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

    altair_saver.save(chart, f'results/neurips_SAC2Q_{env}.pdf')
    charts.append(chart)
alt.vconcat(*charts)

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('finger_all_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('walker_walk_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bice_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
    load_jobs('bbe_all_v3*', subdir='bbe', title='SAC + BBE'),
    load_jobs('ufo_all_v2*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('ufo_all_v3*', subdir='exploration', title='SAC + DEEP'),
    
    load_jobs('pure_explore_behavior_v1*', subdir='exploration', title='SAC + Pure'),
]
data = pd.concat(jobs, sort=False)
# data = data[(data.env == 'ball_in_cup') | (data.env == 'ball_in_cup_explore')]

# N = 500
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
# subset = subset[(subset['episode'] <= N)]

Ns = {
    'ball_in_cup': 500, 'ball_in_cup_explore': 500,
    'reacher': 500, 'reacher_explore': 500,
#     'finger': 1000, 'finger_explore': 1000,
#     'walker': 500, 'walker_explore': 500,
}

names = {
    'ball_in_cup': 'Ball-in-cup', 'ball_in_cup_explore': 'Ball-in-cup explore',
    'reacher': 'Reacher', 'reacher_explore': 'Reacher explore',
}

charts = []
for env in sorted(Ns.keys()):
    N = Ns[env]
    chart = make_base_chart(
        subset[(subset.env == env) & (subset.episode <= N)], 
        title=names[env], 
#         title='',
        color='title',
        window=20,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

#     altair_saver.save(chart, f'results/neurips_pure_{env}.pdf')
    charts.append(chart)
concat_chart = alt.hconcat(*charts).resolve_scale(
    color='independent'
)
altair_saver.save(concat_chart, f'results/neurips_pure_all.png')
concat_chart

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('finger_all_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('walker_walk_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bice_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
    load_jobs('bbe_all_v3*', subdir='bbe', title='SAC + BBE'),
    load_jobs('ufo_all_v2*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('ufo_all_v3*', subdir='exploration', title='SAC + DEEP'),
    
    load_jobs('pure_explore_behavior_v1*', subdir='exploration', title='SAC + Pure'),

]
data = pd.concat(jobs, sort=False)

# Ns = {
#     'ball_in_cup': 500, 'ball_in_cup_explore': 500,
#     'reacher': 500, 'reacher_explore': 500,
#     'finger': 1000, 'finger_explore': 1000,
#     'walker': 500, 'walker_explore': 500,
# }

env_names = {
    'ball_in_cup': 'BIC', 'ball_in_cup_explore': 'BIC explore',
    'reacher': 'Reacher', 'reacher_explore': 'Reacher explore',
    'finger': 'Finger', 'finger_explore': 'Finger explore',
    'walker': 'Walker', 'walker_explore': 'Walker explore',
}

subsets = []
for env in sorted(data.env.unique()):
#     N = Ns[env] / 5
    N = 100    
    env_subset = data[(data.env == env) & (data.test == True) &
                      (data.episode <= N + 10) & (N - 10 < data.episode)].copy()
    env_subset['pretty_env'] = env_names[env]
    subsets.append(env_subset)
subset = pd.concat(subsets, sort=False)
display(subset.groupby(['title', 'pretty_env']).mean().reset_index()[['title', 'pretty_env', 'score', 'episode']])


chart = alt.Chart(subset, width=100, height=300).mark_bar(size=22).encode(
    x=alt.X('title', title='Algorithm', axis=None),
    y=alt.Y('mean(score)', title='Evaluation score', scale=alt.Scale(domain=(0,1000)), axis=alt.Axis(domainOpacity=0)),
    color=alt.Color('title', title='Algorithm', legend=alt.Legend(orient='none', legendX=950, legendY=0)),
    column=alt.Column('pretty_env', title='Environment',
                      sort=['BIC', 'BIC explore', 'Reacher', 'Reacher explore', 'Finger', 'Finger explore', 'Walker', 'Walker explore'],
#                       sort=['Walker', 'Walker explore'],
                      spacing=20, header=alt.Header(labelOrient='bottom', titleOrient='bottom', titleFontSize=24, labelFontSize=20)),
).configure_view(stroke='transparent')


# chart = alt.Chart(subset).mark_bar(opacity=1).encode(
#     column = alt.Column('env:O', spacing = 5, header = alt.Header(labelOrient = "bottom")),
#     x =alt.X('title', sort = ["SAC", "SAC + BBE", "SAC + DEX"],  axis=None),
#     y =alt.Y('mean(score):Q'),
#     color= alt.Color('title')
# ).configure_view(stroke='transparent')

# altair_saver.save(chart, 'results/control_suite_summary.pdf', method='node')
chart.display()

In [ ]:
def load_sunrise_pixels(env_name, task_name):
    df = pd.read_csv(f'results/sunrise/{env_name}-{task_name}.csv')
    df['env'] = env_name
    df['task'] = task_name
    df['title'] = 'SUNRISE'
    df['score'] = df['SUNRISE_avg']
    df['episode'] = df['step'] / 1000
    df['test'] = True
    df['name'] = 'SAC'
    return df

def load_sunrise_rad_pixels(env_name, task_name):
    df = pd.read_csv(f'results/sunrise_rad/{env_name}-{task_name}.csv')
    df['env'] = env_name
    df['task'] = task_name
    df['title'] = 'SUNRISE RAD'
    df['score'] = df['SUNRISE_avg']
    df['episode'] = df['step'] / 1000
    df['test'] = True
    df['name'] = 'SAC'
    return df

In [ ]:
def load_sunrise(env_name, task_name):
    dfs = []
    for filename in glob.glob(f'/home/will/code/sunrise/OpenAIGym_SAC/data/{env_name}.{task_name}/SUNRISE/*/progress.csv'):
        raw_df = pd.read_csv(filename)
        clean_df = pd.DataFrame({
            'step': raw_df['evaluation/num steps total'],
            'episode': raw_df['evaluation/num steps total'] / 1000,
            'score': raw_df['evaluation/Returns Mean'],
        })
        seed_ish = len(dfs)
        clean_df['name'] = f'SUNRISE_seed{seed_ish}'
        clean_df['title'] = 'SUNRISE'
        clean_df['env'] = env_name
        clean_df['task'] = task_name
        clean_df['test'] = True
        clean_df['seed'] = seed_ish
        dfs.append(clean_df)
    return pd.concat(dfs, sort=False).reset_index(drop=True)
    
df = load_sunrise('ball_in_cup', 'catch')

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('finger_all_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('walker_walk_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bice_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
    load_jobs('bbe_all_v3*', subdir='bbe', title='SAC + BBE'),
    load_jobs('ufo_all_v2*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('ufo_all_v3*', subdir='exploration', title='SAC + DEEP'),
    
#     load_jobs('pure_explore_behavior_v1*', subdir='exploration', title='SAC + Pure'),
    load_sunrise('ball_in_cup', 'catch'),
    load_sunrise('ball_in_cup_explore', 'catch'),
    load_sunrise('reacher', 'hard'),
    load_sunrise('reacher_explore', 'hard_narrow_init'),
]
data = pd.concat(jobs, sort=False)

N = 500
subset = data
subset = subset[(subset['episode'] <= N)]

Ns = {
    'ball_in_cup': 500, 'ball_in_cup_explore': 500,
    'reacher': 500, 'reacher_explore': 500,
#     'finger': 1000, 'finger_explore': 1000,
#     'walker': 500, #'walker_explore': 500,
}

names = {
    'ball_in_cup': 'Ball-in-cup', 'ball_in_cup_explore': 'Ball-in-cup explore',
    'reacher': 'Reacher', 'reacher_explore': 'Reacher explore',
    'finger': 'Finger', 'finger_explore': 'Finger Explore',
    'walker': 'Walker', 'walker_explore': 'Walker Explore'
}

charts = []
for env in sorted(Ns.keys()):
    N = Ns[env]
    chart = make_base_chart(
        subset[(subset.env == env) & (subset.episode <= N)], 
        title=names[env], 
#         title='',
        color='title',
        window=20,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

#     altair_saver.save(chart, f'results/neurips_pure_{env}.pdf')
    charts.append(chart)
concat_chart = alt.hconcat(*charts).resolve_scale(
    color='independent'
)
altair_saver.save(concat_chart, f'results/neurips_sunrise_all.png')
concat_chart

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('finger_all_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('walker_walk_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bice_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
    load_jobs('bbe_all_v3*', subdir='bbe', title='SAC + BBE'),
    load_jobs('ufo_all_v2*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('ufo_all_v3*', subdir='exploration', title='SAC + DEEP'),
    
    load_sunrise('ball_in_cup', 'catch'),
    load_sunrise('ball_in_cup_explore', 'catch'),
    load_sunrise('reacher', 'hard'),
    load_sunrise('reacher_explore', 'hard_narrow_init'),
]
data = pd.concat(jobs, sort=False)

# Ns = {
#     'ball_in_cup': 500, 'ball_in_cup_explore': 500,
#     'reacher': 500, 'reacher_explore': 500,
#     'finger': 1000, 'finger_explore': 1000,
#     'walker': 500, 'walker_explore': 500,
# }

env_names = {
    'ball_in_cup': 'BIC', 'ball_in_cup_explore': 'BIC explore',
    'reacher': 'Reacher', 'reacher_explore': 'Reacher explore',
    'finger': 'Finger', 'finger_explore': 'Finger explore',
    'walker': 'Walker', 'walker_explore': 'Walker explore',
}

subsets = []
for env in sorted(data.env.unique()):
#     N = Ns[env] / 5
    N = 100    
    env_subset = data[(data.env == env) & (data.test == True) &
                      (data.episode <= N + 10) & (N - 10 < data.episode)].copy()
    env_subset['pretty_env'] = env_names[env]
    subsets.append(env_subset)
subset = pd.concat(subsets, sort=False)
display(subset.groupby(['title', 'pretty_env']).mean().reset_index()[['title', 'pretty_env', 'score', 'episode']])

chart = alt.Chart(subset, width=100, height=300).mark_bar(size=22).encode(
    x=alt.X('title', title='Algorithm', axis=None),
    y=alt.Y('mean(score)', title='Evaluation score', scale=alt.Scale(domain=(0,1000)), axis=alt.Axis(domainOpacity=0)),
    color=alt.Color('title', title='Algorithm', legend=alt.Legend(orient='none', legendX=950, legendY=0)),
    column=alt.Column('pretty_env', title='Environment',
                      sort=['BIC', 'BIC explore', 'Reacher', 'Reacher explore', 'Finger', 'Finger explore', 'Walker', 'Walker explore'],
#                       sort=['Walker', 'Walker explore'],
                      spacing=20, header=alt.Header(labelOrient='bottom', titleOrient='bottom', titleFontSize=24, labelFontSize=20)),
).configure_view(stroke='transparent')


# chart = alt.Chart(subset).mark_bar(opacity=1).encode(
#     column = alt.Column('env:O', spacing = 5, header = alt.Header(labelOrient = "bottom")),
#     x =alt.X('title', sort = ["SAC", "SAC + BBE", "SAC + DEX"],  axis=None),
#     y =alt.Y('mean(score):Q'),
#     color= alt.Color('title')
# ).configure_view(stroke='transparent')

# altair_saver.save(chart, 'results/control_suite_summary.pdf', method='node')
chart.display()

## Ablations

In [ ]:
alt.themes.enable('publication')
jobs = [
#     load_jobs('ufo_all_v2*', subdir='exploration'),
#     load_jobs('ufo_all_v3*', subdir='exploration'),

    load_jobs('ablation_v4_sanity*', subdir='exploration'),
    load_jobs('ablation_v1*', subdir='exploration'),
    load_jobs('ablation_v2_optimism*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)
subset = data

Ns = {
#     'ball_in_cup': 500, 
    'ball_in_cup_explore': 100,
#     'reacher': 500, 'reacher_explore': 500,
#     'finger': 1000, 'finger_explore': 1000,
#     'walker': 500, #'walker_explore': 500,
}

names = {
    'ball_in_cup': 'Ball-in-cup', 'ball_in_cup_explore': 'Ball-in-cup explore',
    'reacher': 'Reacher', 'reacher_explore': 'Reacher explore',
    'finger': 'Finger', 'finger_explore': 'Finger Explore',
    'walker': 'Walker', 'walker_explore': 'Walker Explore'
}

subset['Temperature'] = subset['temperature']

subset['Candidate actions'] = subset['n_candidates']
subset['Candidate actions'] = subset['Candidate actions'].fillna(64)

subset['Optimism'] = subset['optimistic_actions']

defaults = {
    'Temperature': 1e-1,
    'Candidate actions': 64,
    'Optimism': True,
}

env = 'ball_in_cup_explore'
charts = []
for ablation in ['Temperature', 'Candidate actions', 'Optimism']:
    
    chart_subset = subset.copy()
    for other_ablation in ['Temperature', 'Candidate actions', 'Optimism']:
        if ablation != other_ablation:
            chart_subset = chart_subset[chart_subset[other_ablation] == defaults[other_ablation]]
            
    N = Ns[env]
    chart = make_base_chart(
        chart_subset[(chart_subset.env == env) & (chart_subset.episode <= N)], 
        title=names[env], 
        color=ablation + ':O',
        window=20,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color(ablation + ':O', legend=alt.Legend(orient='bottom-right')), strip=False)

#     altair_saver.save(chart, f'results/neurips_ablation_{ablation}.pdf')
    charts.append(chart)
bic_concat_chart = alt.hconcat(*charts).resolve_scale(
    color='independent'
)
bic_concat_chart

In [ ]:
alt.themes.enable('publication')
jobs = [
#     load_jobs('ufo_all_v2*', subdir='exploration'),
#     load_jobs('ufo_all_v3*', subdir='exploration'),

#     load_jobs('ablation_v3_reacher*', subdir='exploration'),
#     load_jobs('ablation_v2_optimism*', subdir='exploration'),
    load_jobs('ablation_v4_sanity*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)
subset = data

Ns = {
#     'ball_in_cup': 500, 
#     'ball_in_cup_explore': 100,
#     'reacher': 500, 
    'reacher_explore': 150,
#     'finger': 1000, 'finger_explore': 1000,
#     'walker': 500, #'walker_explore': 500,
}

names = {
    'ball_in_cup': 'Ball-in-cup', 'ball_in_cup_explore': 'Ball-in-cup explore',
    'reacher': 'Reacher', 'reacher_explore': 'Reacher explore',
    'finger': 'Finger', 'finger_explore': 'Finger Explore',
    'walker': 'Walker', 'walker_explore': 'Walker Explore'
}

subset['Temperature'] = subset['temperature']

subset['Candidate actions'] = subset['n_candidates']
subset['Candidate actions'] = subset['Candidate actions'].fillna(64)

subset['Optimism'] = subset['optimistic_actions']

defaults = {
    'Temperature': 1e-1,
    'Candidate actions': 64,
    'Optimism': True,
}

env = 'reacher_explore'
charts = []
for ablation in ['Temperature', 'Candidate actions', 'Optimism']:
    
    chart_subset = subset.copy()
    for other_ablation in ['Temperature', 'Candidate actions', 'Optimism']:
        if ablation != other_ablation:
            chart_subset = chart_subset[chart_subset[other_ablation] == defaults[other_ablation]]

    N = Ns[env]
    chart = make_base_chart(
        chart_subset[(chart_subset.env == env) & (chart_subset.episode <= N)], 
        title=names[env], 
        color=ablation + ':O',
        window=20,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color(ablation + ':O', legend=alt.Legend(orient='bottom-right')), strip=False)

#     altair_saver.save(chart, f'results/neurips_ablation_{ablation}.pdf')
    charts.append(chart)
reacher_concat_chart = alt.hconcat(*charts).resolve_scale(
    color='independent'
)
reacher_concat_chart

In [ ]:
bic_concat_chart & reacher_concat_chart